In [1]:
"""
Automatic speech recognition and speaker identity
Using conv1D shared, but doesn't seem to work
"""
from __future__ import division, print_function, absolute_import
import os
import numpy as np
import sklearn.preprocessing as prep
import librosa
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense, Dropout
from tensorflow.keras.layers import Embedding, Bidirectional,Conv1D,GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import utils,losses

In [2]:
print("Imports doned")
def wav2mfcc(file_path, max_pad_len=30):
    wave, sampleRate = librosa.load(file_path, mono=True, sr=None)
    wave=prep.maxabs_scale(wave) #Normalize amplitude to +-1
    mfcc = librosa.feature.mfcc(wave, sr=sampleRate).transpose()
    print(sampleRate,"wave=",wave.shape,"mfcc=",mfcc.shape,"file=",file_path)
    #print(mfcc[1,:5])
    dif=max_pad_len-mfcc.shape[0]
    #padding sample
    if dif>0:
        #fill holes
        #print("filling holes")
        for i in range(dif):
            mfcc=np.vstack([mfcc,np.zeros(mfcc.shape[1])])
    else:
        #take away excess
        #print("excess")
        mfcc=mfcc[:max_pad_len]
    #print(mfcc.shape)
    #print(mfcc[199,:5])
    return mfcc

# load wav files with this format speaker_word_number.wav
class EasyASR(object):
    '''
    classdocs
    '''
    def __init__(self, filePath="/home/francisco/voz"):
        '''
        Constructor
        '''
        basepath=filePath
        self.mfccs=[]
        self.speakers=[]
        self.words=[]
        fNames=os.listdir(basepath)
        print("fNames=",len(fNames))
        for i,fn in enumerate(fNames):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            mfccs=wav2mfcc(basepath+'/{}.wav'.format(n))
            self.mfccs.append(mfccs)
            #decode file name format speaker_word_number
            speaker,word,number=n.split("_")
            self.speakers.append(speaker)
            self.words.append(word)
        self.leSpeakers=prep.LabelEncoder()
        self.leSpeakers.fit(self.speakers)
        self.iSpeakerLabels=self.leSpeakers.transform(self.speakers)
        self.leWords=prep.LabelEncoder()
        self.leWords.fit(self.words)
        self.iWordLabels=self.leWords.transform(self.words)
        print(len(self.words))
        print(self.words)
        print(self.speakers)

ea=EasyASR()
print(ea.leWords)

Imports doned
fNames= 108
16000 wave= (3871,) mfcc= (8, 20) file= /home/francisco/voz/paco_no_006.wav
16000 wave= (4963,) mfcc= (10, 20) file= /home/francisco/voz/paco_si_008.wav
16000 wave= (9285,) mfcc= (19, 20) file= /home/francisco/voz/paco_dos_002.wav
16000 wave= (4434,) mfcc= (9, 20) file= /home/francisco/voz/paco_uno_004.wav
16000 wave= (2428,) mfcc= (5, 20) file= /home/francisco/voz/paco_no_007.wav
16000 wave= (4882,) mfcc= (10, 20) file= /home/francisco/voz/paco_para_003.wav
16000 wave= (6972,) mfcc= (14, 20) file= /home/francisco/voz/paco_cuatro_003.wav
16000 wave= (6524,) mfcc= (13, 20) file= /home/francisco/voz/paco_nueve_001.wav
16000 wave= (4630,) mfcc= (10, 20) file= /home/francisco/voz/carlos_si_002.wav
16000 wave= (14450,) mfcc= (29, 20) file= /home/francisco/voz/paco_buenos-dias_001.wav
16000 wave= (14816,) mfcc= (29, 20) file= /home/francisco/voz/paco_buenos-dias_003.wav
16000 wave= (6847,) mfcc= (14, 20) file= /home/francisco/voz/irene_hola_003.wav
16000 wave= (9176

In [3]:
trainX, trainYwords, trainYspeakers = np.array(ea.mfccs),np.array(ea.iWordLabels),np.array(ea.iSpeakerLabels)
#testX, testY = test

maxlen=trainX.shape[1]
lstm_hidden_units=128
input_dim=trainX.shape[2]
outputWords_dim=max(trainYwords)+1
outputSpeakers_dim=max(trainYspeakers)+1

print(outputWords_dim,outputSpeakers_dim)

# Converting labels to binary vectors
trainYwords_cat    = utils.to_categorical(trainYwords   ,outputWords_dim)
trainYspeakers_cat = utils.to_categorical(trainYspeakers,outputSpeakers_dim)
print (trainX.shape,trainYwords_cat.shape,trainYspeakers_cat.shape)

17 4
(108, 30, 20) (108, 17) (108, 4)


In [43]:
# Network building
def build_model_asr_conv1d(input_dim,outputWords_dim,outputSpeakers_dim,hiden_dim=20):
    input = Input(shape=(None,input_dim))

    # Conv1D + global max pooling
    conv1d=Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=2)
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(input)
    x = Dropout(0.5)(x)
    x =conv1d(x)
    #x = Dropout(0.5)(x)
    #x = conv1d(x)
    #x = Dropout(0.5)(x)
    #x = conv1d(x)
    #x = Dropout(0.5)(x)
    #x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(x)
    x = GlobalMaxPooling1D()(x)

    # We add a vanilla hidden layer:
    x = Dense(hiden_dim, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(hiden_dim)(x)

    # from: https://www.tensorflow.org/guide/keras/train_and_evaluate
    #       Passing data to multi-input, multi-output models
    prediction_words    = Dense(outputWords_dim   , activation="softmax", name="word")(x)
    prediction_speakers = Dense(outputSpeakers_dim, activation="softmax", name="speaker")(x)

    model = Model(input, [prediction_words,prediction_speakers])
    return model

In [44]:
model=build_model_asr_conv1d(input_dim,outputWords_dim,outputSpeakers_dim)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, None, 20)]   0                                            
__________________________________________________________________________________________________
conv1d_16 (Conv1D)              (None, None, 20)     1220        input_8[0][0]                    
__________________________________________________________________________________________________
dropout_23 (Dropout)            (None, None, 20)     0           conv1d_16[0][0]                  
__________________________________________________________________________________________________
conv1d_15 (Conv1D)              (None, None, 20)     1220        dropout_23[0][0]                 
____________________________________________________________________________________________

In [45]:
# Training
history=model.fit(trainX, [trainYwords_cat,trainYspeakers_cat], batch_size=108,epochs=5000,validation_split=0.2)

Train on 86 samples, validate on 22 samples
Epoch 1/5000
86/86 [==============================] - 1s 7ms/sample - loss: 92.8915 - word_loss: 62.9080 - speaker_loss: 29.9836 - word_accuracy: 0.0465 - speaker_accuracy: 0.2326 - val_loss: 31.4003 - val_word_loss: 26.6230 - val_speaker_loss: 4.7772 - val_word_accuracy: 0.0455 - val_speaker_accuracy: 0.5909
Epoch 2/5000
86/86 [==============================] - 0s 147us/sample - loss: 82.6090 - word_loss: 53.2628 - speaker_loss: 29.3462 - word_accuracy: 0.1279 - speaker_accuracy: 0.2442 - val_loss: 28.6350 - val_word_loss: 24.8552 - val_speaker_loss: 3.7799 - val_word_accuracy: 0.0455 - val_speaker_accuracy: 0.6818
Epoch 3/5000
86/86 [==============================] - 0s 138us/sample - loss: 78.3415 - word_loss: 53.0436 - speaker_loss: 25.2978 - word_accuracy: 0.0930 - speaker_accuracy: 0.2791 - val_loss: 26.7031 - val_word_loss: 23.5834 - val_speaker_loss: 3.1198 - val_word_accuracy: 0.0455 - val_speaker_accuracy: 0.6818
Epoch 4/5000
86/86 

Epoch 28/5000
86/86 [==============================] - 0s 162us/sample - loss: 16.4495 - word_loss: 13.4525 - speaker_loss: 2.9969 - word_accuracy: 0.1047 - speaker_accuracy: 0.5698 - val_loss: 7.9313 - val_word_loss: 6.6119 - val_speaker_loss: 1.3194 - val_word_accuracy: 0.0909 - val_speaker_accuracy: 0.8636
Epoch 29/5000
86/86 [==============================] - 0s 178us/sample - loss: 16.4027 - word_loss: 13.4066 - speaker_loss: 2.9961 - word_accuracy: 0.1047 - speaker_accuracy: 0.5814 - val_loss: 7.6530 - val_word_loss: 6.3423 - val_speaker_loss: 1.3107 - val_word_accuracy: 0.0909 - val_speaker_accuracy: 0.8636
Epoch 30/5000
86/86 [==============================] - 0s 183us/sample - loss: 16.4145 - word_loss: 11.0893 - speaker_loss: 5.3251 - word_accuracy: 0.1279 - speaker_accuracy: 0.4651 - val_loss: 7.3999 - val_word_loss: 6.0916 - val_speaker_loss: 1.3084 - val_word_accuracy: 0.0909 - val_speaker_accuracy: 0.8636
Epoch 31/5000
86/86 [==============================] - 0s 200us/sam

Epoch 55/5000
86/86 [==============================] - 0s 166us/sample - loss: 7.6972 - word_loss: 5.6623 - speaker_loss: 2.0349 - word_accuracy: 0.1047 - speaker_accuracy: 0.5814 - val_loss: 4.5491 - val_word_loss: 3.3613 - val_speaker_loss: 1.1879 - val_word_accuracy: 0.0909 - val_speaker_accuracy: 0.8182
Epoch 56/5000
86/86 [==============================] - 0s 162us/sample - loss: 6.3136 - word_loss: 4.4435 - speaker_loss: 1.8701 - word_accuracy: 0.1860 - speaker_accuracy: 0.5465 - val_loss: 4.5227 - val_word_loss: 3.3334 - val_speaker_loss: 1.1893 - val_word_accuracy: 0.0909 - val_speaker_accuracy: 0.8182
Epoch 57/5000
86/86 [==============================] - 0s 157us/sample - loss: 6.6613 - word_loss: 5.5542 - speaker_loss: 1.1071 - word_accuracy: 0.0930 - speaker_accuracy: 0.6860 - val_loss: 4.4939 - val_word_loss: 3.3046 - val_speaker_loss: 1.1893 - val_word_accuracy: 0.0909 - val_speaker_accuracy: 0.8182
Epoch 58/5000
86/86 [==============================] - 0s 157us/sample - 

Epoch 82/5000
86/86 [==============================] - 0s 170us/sample - loss: 4.9872 - word_loss: 4.0357 - speaker_loss: 0.9515 - word_accuracy: 0.1860 - speaker_accuracy: 0.7326 - val_loss: 4.1560 - val_word_loss: 2.9607 - val_speaker_loss: 1.1953 - val_word_accuracy: 0.0909 - val_speaker_accuracy: 0.8182
Epoch 83/5000
86/86 [==============================] - 0s 144us/sample - loss: 4.5371 - word_loss: 3.5107 - speaker_loss: 1.0265 - word_accuracy: 0.1395 - speaker_accuracy: 0.6628 - val_loss: 4.1529 - val_word_loss: 2.9566 - val_speaker_loss: 1.1963 - val_word_accuracy: 0.0909 - val_speaker_accuracy: 0.8182
Epoch 84/5000
86/86 [==============================] - 0s 137us/sample - loss: 5.9958 - word_loss: 4.3714 - speaker_loss: 1.6244 - word_accuracy: 0.1279 - speaker_accuracy: 0.5814 - val_loss: 4.1464 - val_word_loss: 2.9537 - val_speaker_loss: 1.1926 - val_word_accuracy: 0.0909 - val_speaker_accuracy: 0.8182
Epoch 85/5000
86/86 [==============================] - 0s 144us/sample - 

Epoch 109/5000
86/86 [==============================] - 0s 148us/sample - loss: 3.5440 - word_loss: 2.7757 - speaker_loss: 0.7683 - word_accuracy: 0.1744 - speaker_accuracy: 0.7674 - val_loss: 3.9642 - val_word_loss: 2.8826 - val_speaker_loss: 1.0816 - val_word_accuracy: 0.0455 - val_speaker_accuracy: 0.8182
Epoch 110/5000
86/86 [==============================] - 0s 133us/sample - loss: 4.0248 - word_loss: 3.1037 - speaker_loss: 0.9210 - word_accuracy: 0.1163 - speaker_accuracy: 0.7442 - val_loss: 3.9562 - val_word_loss: 2.8793 - val_speaker_loss: 1.0769 - val_word_accuracy: 0.0455 - val_speaker_accuracy: 0.8182
Epoch 111/5000
86/86 [==============================] - 0s 140us/sample - loss: 4.1205 - word_loss: 3.2035 - speaker_loss: 0.9171 - word_accuracy: 0.1512 - speaker_accuracy: 0.7209 - val_loss: 3.9479 - val_word_loss: 2.8757 - val_speaker_loss: 1.0722 - val_word_accuracy: 0.0455 - val_speaker_accuracy: 0.8182
Epoch 112/5000
86/86 [==============================] - 0s 145us/sampl

...

Epoch 4888/5000
86/86 [==============================] - 0s 145us/sample - loss: 1.1005 - word_loss: 0.8902 - speaker_loss: 0.2104 - word_accuracy: 0.6744 - speaker_accuracy: 0.9302 - val_loss: 1.7993 - val_word_loss: 1.2602 - val_speaker_loss: 0.5391 - val_word_accuracy: 0.6364 - val_speaker_accuracy: 0.8636
Epoch 4889/5000
86/86 [==============================] - 0s 151us/sample - loss: 1.1411 - word_loss: 0.8969 - speaker_loss: 0.2442 - word_accuracy: 0.7326 - speaker_accuracy: 0.9186 - val_loss: 1.7579 - val_word_loss: 1.2549 - val_speaker_loss: 0.5030 - val_word_accuracy: 0.6364 - val_speaker_accuracy: 0.8636
Epoch 4890/5000
86/86 [==============================] - 0s 150us/sample - loss: 0.8362 - word_loss: 0.6922 - speaker_loss: 0.1440 - word_accuracy: 0.7558 - speaker_accuracy: 0.9419 - val_loss: 1.7149 - val_word_loss: 1.2402 - val_speaker_loss: 0.4747 - val_word_accuracy: 0.6364 - val_speaker_accuracy: 0.9091
Epoch 4891/5000
86/86 [==============================] - 0s 154us/s

Epoch 4915/5000
86/86 [==============================] - 0s 149us/sample - loss: 0.9116 - word_loss: 0.7882 - speaker_loss: 0.1234 - word_accuracy: 0.7209 - speaker_accuracy: 0.9651 - val_loss: 1.6226 - val_word_loss: 1.1429 - val_speaker_loss: 0.4796 - val_word_accuracy: 0.6364 - val_speaker_accuracy: 0.9091
Epoch 4916/5000
86/86 [==============================] - 0s 181us/sample - loss: 0.7748 - word_loss: 0.5768 - speaker_loss: 0.1980 - word_accuracy: 0.7674 - speaker_accuracy: 0.9302 - val_loss: 1.6366 - val_word_loss: 1.1432 - val_speaker_loss: 0.4934 - val_word_accuracy: 0.6364 - val_speaker_accuracy: 0.8636
Epoch 4917/5000
86/86 [==============================] - 0s 154us/sample - loss: 0.9288 - word_loss: 0.7944 - speaker_loss: 0.1343 - word_accuracy: 0.7674 - speaker_accuracy: 0.9651 - val_loss: 1.6460 - val_word_loss: 1.1430 - val_speaker_loss: 0.5031 - val_word_accuracy: 0.6364 - val_speaker_accuracy: 0.8636
Epoch 4918/5000
86/86 [==============================] - 0s 167us/s

Epoch 4942/5000
86/86 [==============================] - 0s 158us/sample - loss: 0.6190 - word_loss: 0.5085 - speaker_loss: 0.1106 - word_accuracy: 0.8140 - speaker_accuracy: 0.9651 - val_loss: 1.5211 - val_word_loss: 1.0842 - val_speaker_loss: 0.4369 - val_word_accuracy: 0.6818 - val_speaker_accuracy: 0.9091
Epoch 4943/5000
86/86 [==============================] - 0s 145us/sample - loss: 0.7564 - word_loss: 0.6328 - speaker_loss: 0.1236 - word_accuracy: 0.7558 - speaker_accuracy: 0.9419 - val_loss: 1.5146 - val_word_loss: 1.0545 - val_speaker_loss: 0.4601 - val_word_accuracy: 0.6818 - val_speaker_accuracy: 0.8636
Epoch 4944/5000
86/86 [==============================] - 0s 150us/sample - loss: 0.9190 - word_loss: 0.8054 - speaker_loss: 0.1136 - word_accuracy: 0.7093 - speaker_accuracy: 0.9535 - val_loss: 1.5351 - val_word_loss: 1.0319 - val_speaker_loss: 0.5032 - val_word_accuracy: 0.6364 - val_speaker_accuracy: 0.8182
Epoch 4945/5000
86/86 [==============================] - 0s 146us/s

Epoch 4969/5000
86/86 [==============================] - 0s 169us/sample - loss: 0.7262 - word_loss: 0.5993 - speaker_loss: 0.1269 - word_accuracy: 0.8140 - speaker_accuracy: 0.9535 - val_loss: 1.6745 - val_word_loss: 1.1738 - val_speaker_loss: 0.5007 - val_word_accuracy: 0.6818 - val_speaker_accuracy: 0.8636
Epoch 4970/5000
86/86 [==============================] - 0s 181us/sample - loss: 1.1115 - word_loss: 0.8314 - speaker_loss: 0.2801 - word_accuracy: 0.7093 - speaker_accuracy: 0.9302 - val_loss: 1.6502 - val_word_loss: 1.1532 - val_speaker_loss: 0.4969 - val_word_accuracy: 0.6818 - val_speaker_accuracy: 0.8636
Epoch 4971/5000
86/86 [==============================] - 0s 163us/sample - loss: 0.7683 - word_loss: 0.6524 - speaker_loss: 0.1159 - word_accuracy: 0.7791 - speaker_accuracy: 0.9651 - val_loss: 1.6316 - val_word_loss: 1.1389 - val_speaker_loss: 0.4927 - val_word_accuracy: 0.6818 - val_speaker_accuracy: 0.8636
Epoch 4972/5000
86/86 [==============================] - 0s 156us/s

Epoch 4996/5000
86/86 [==============================] - 0s 160us/sample - loss: 0.6973 - word_loss: 0.6262 - speaker_loss: 0.0712 - word_accuracy: 0.8023 - speaker_accuracy: 0.9651 - val_loss: 1.5619 - val_word_loss: 1.0933 - val_speaker_loss: 0.4686 - val_word_accuracy: 0.6818 - val_speaker_accuracy: 0.8636
Epoch 4997/5000
86/86 [==============================] - 0s 161us/sample - loss: 0.7157 - word_loss: 0.5598 - speaker_loss: 0.1559 - word_accuracy: 0.8140 - speaker_accuracy: 0.9535 - val_loss: 1.5116 - val_word_loss: 1.0608 - val_speaker_loss: 0.4508 - val_word_accuracy: 0.6818 - val_speaker_accuracy: 0.8636
Epoch 4998/5000
86/86 [==============================] - 0s 154us/sample - loss: 1.0825 - word_loss: 0.8755 - speaker_loss: 0.2070 - word_accuracy: 0.7558 - speaker_accuracy: 0.9186 - val_loss: 1.4618 - val_word_loss: 1.0306 - val_speaker_loss: 0.4312 - val_word_accuracy: 0.6818 - val_speaker_accuracy: 0.9091
Epoch 4999/5000
86/86 [==============================] - 0s 154us/s

In [46]:
print(trainX[5:7,:,:].shape)
predWord,predSpeaker=model.predict(trainX[5:6,:,:])
print(predWord,predSpeaker)
y_predWord   = np.argmax(predWord)
y_predSpeaker= np.argmax(predSpeaker)
print(y_predWord,"=",np.max(predWord))
print(y_predSpeaker,"=",np.max(predSpeaker))
word=ea.leWords.inverse_transform([y_predWord])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
print(speaker[0]," ha dicho \'",word[0],"\'")
print(ea.leWords.classes_)
print(ea.leSpeakers.classes_)

(2, 30, 20)
[[1.0131357e-06 2.1762084e-08 1.6787067e-02 3.1755611e-21 3.7724800e-07
  1.0044962e-23 3.2635369e-16 1.6766863e-15 1.2376901e-09 2.2580673e-25
  9.7874135e-01 4.3549626e-03 1.8565805e-08 5.8241335e-20 5.4929661e-08
  1.1511929e-04 2.0160212e-31]] [[4.0688142e-21 1.1910344e-18 1.5771326e-15 1.0000000e+00]]
10 = 0.97874135
3 = 1.0
paco  ha dicho ' para '
['avanza' 'buenos-dias' 'casa' 'cinco' 'cuatro' 'dos' 'hola' 'no' 'nueve'
 'ocho' 'para' 'retrocede' 'seis' 'si' 'siete' 'tres' 'uno']
['carlos' 'eva' 'irene' 'paco']
